# Example of Neural Network Model
This is an example of a simple implementation of a neural network model. Similar, but not equal to 

> Bengio, Y., Ducharme, R., Vincent, P., & Jauvin, C. (2003). A neural probabilistic language model. *Journal of machine learning research*, *3*(Feb), 1137-1155

In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm_notebook

In [2]:
from langmodels.corpora.moviedialog import MovieDialogCollection

In [3]:
db_name = 'movie-dialogs'
collection = 'lines'
m = {'$match': {'character.movie.id': 'm42'}}
p = {'$project': {'_id': 0, 'id': 1, 'text': 1}}
pipeline = [m, p]

In [4]:
m42 = MovieDialogCollection(db_name, collection, 
                              use_pos=False, pipeline=pipeline)

In [5]:
V = m42.vocabulary

## Indexing words and create sequences

In [6]:
idx2w = {i:w for (i, w) in enumerate(V)}
w2idx = {w:i for (i, w) in enumerate(V)}

In [7]:
from keras.preprocessing.sequence import pad_sequences

Using TensorFlow backend.


In [15]:
sequences = []
for doc, tokens in m42.get_tokens():
    sequences.append([w2idx[t] for t in tokens])
sequences = np.array(pad_sequences(sequences, padding='pre'))

## Create training and target labels

In [16]:
from keras.utils import to_categorical

In [17]:
X, y = sequences[:,:-1], sequences[:,-1]

In [19]:
y = to_categorical(y, num_classes=len(V))

In [20]:
print(X.shape, y.shape)

(834, 106) (834, 1363)


## Define the embedding layer
The embedding layer is based on the size of the vocabulary and the length of input sequences, plus the number of dimensions for vectors.

In [25]:
from keras.layers import Embedding, LSTM, Dense
from keras.models import Sequential

In [22]:
embedding_size = 50
E = Embedding(len(V), embedding_size, input_length=X.shape[1])

## Network architecture

In [29]:
model = Sequential()
model.add(E)
model.add(LSTM(100, return_sequences=False))
model.add(Dense(100, activation='relu'))
model.add(Dense(len(V), activation='softmax'))

In [30]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 106, 50)           68150     
_________________________________________________________________
lstm_2 (LSTM)                (None, 100)               60400     
_________________________________________________________________
dense_3 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_4 (Dense)              (None, 1363)              137663    
Total params: 276,313
Trainable params: 276,313
Non-trainable params: 0
_________________________________________________________________


## Train

In [31]:
model.compile(loss='categorical_crossentropy', optimizer='adam', 
              metrics=['accuracy'])
model.fit(X, y, batch_size=128, epochs=100)

/Users/alfio/Applications/anaconda3/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/100
834/834 [==============================] - 2s 2ms/step - loss: 7.2135 - accuracy: 0.0156
Epoch 2/100
834/834 [==============================] - 1s 1ms/step - loss: 7.1450 - accuracy: 0.0300
Epoch 3/100
834/834 [==============================] - 1s 1ms/step - loss: 6.5437 - accuracy: 0.0240
Epoch 4/100
834/834 [==============================] - 1s 1ms/step - loss: 5.9276 - accuracy: 0.0240
Epoch 5/100
834/834 [==============================] - 1s 1ms/step - loss: 5.7645 - accuracy: 0.0408
Epoch 6/100
834/834 [==============================] - 1s 1ms/step - loss: 5.6573 - accuracy: 0.0408
Epoch 7/100
834/834 [==============================] - 1s 1ms/step - loss: 5.6330 - accuracy: 0.0408
Epoch 8/100
834/834 [==============================] - 1s 1ms/step - loss: 5.6102 - accuracy: 0.0408
Epoch 9/100
834/834 [==============================] - 1s 1ms/step - loss: 5.5990 - accuracy: 0.0408
Epoch 10/100
834/834 [==============================] - 1s 1ms/step - loss: 5.5933 - accura

834/834 [==============================] - 1s 1ms/step - loss: 2.3499 - accuracy: 0.4173
Epoch 82/100
834/834 [==============================] - 1s 1ms/step - loss: 2.3119 - accuracy: 0.4077
Epoch 83/100
834/834 [==============================] - 1s 1ms/step - loss: 2.2667 - accuracy: 0.4257
Epoch 84/100
834/834 [==============================] - 1s 1ms/step - loss: 2.2205 - accuracy: 0.4712
Epoch 85/100
834/834 [==============================] - 1s 1ms/step - loss: 2.1819 - accuracy: 0.4580
Epoch 86/100
834/834 [==============================] - 1s 1ms/step - loss: 2.1249 - accuracy: 0.4916
Epoch 87/100
834/834 [==============================] - 1s 1ms/step - loss: 2.0740 - accuracy: 0.4808
Epoch 88/100
834/834 [==============================] - 1s 1ms/step - loss: 2.0367 - accuracy: 0.5144
Epoch 89/100
834/834 [==============================] - 1s 1ms/step - loss: 2.0040 - accuracy: 0.5300
Epoch 90/100
834/834 [==============================] - 1s 1ms/step - loss: 1.9567 - accuracy: 

## Predict

In [38]:
test = sequences[42]
print([idx2w[x] for x in test if x != 0])
ask = np.array([0] + test[:-1])
print([idx2w[x] for x in ask if x != 0])

['we', 'read', 'five', 'times', 'that', 'you', 'were', 'killed', 'in', 'five', 'different', 'places']
['we', 'read', 'five', 'times', 'that', 'you', 'were', 'killed', 'in', 'five', 'different']


In [49]:
w = model.predict_classes(ask.reshape(1, -1))
print(w)

[202]


In [50]:
model.predict(ask.reshape(1, -1))

array([[2.7549209e-15, 5.3787658e-17, 5.2095036e-16, ..., 1.1419303e-15,
        5.3773733e-16, 7.4884436e-17]], dtype=float32)

In [51]:
print(idx2w[w[0]])

places


## Assess the word embeddings

In [55]:
E.embeddings.shape

TensorShape([1363, 50])